In [3]:
!pip install pandas==1.3.0
!pip install matplotlib==3.3.3

ERROR: Could not find a version that satisfies the requirement pandas==1.3.0 (from versions: none)
ERROR: No matching distribution found for pandas==1.3.0
ERROR: Could not find a version that satisfies the requirement matplotlib==3.3.3 (from versions: none)
ERROR: No matching distribution found for matplotlib==3.3.3


In [18]:
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
import os
from torchvision.io import read_image
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import torch.cuda.amp as amp
import torch.optim as optim
from PIL import Image
from collections import OrderedDict
%matplotlib inline

# Load_train&valid_dataset

In [2]:
root = '../input/d2lclassifyleaves'
t_data_raw = pd.read_csv(root+'/'+'train.csv',index_col='image')
t_data_raw['label_index'] = torch.zeros((t_data_raw.shape[0])).type(torch.int32).numpy()
label_maps = {}
for (i, label) in enumerate(t_data_raw.label.unique()):
    label_maps[i] = label
    t_data_raw.loc[t_data_raw.label == label, 'label_index'] = i
t_data_raw.to_csv('./'+'trainindex.csv')
t_data_raw.shape
t_data_raw

,label,label_index
image,,
images/0.jpg,maclura_pomifera,0
images/1.jpg,maclura_pomifera,0
images/2.jpg,maclura_pomifera,0
images/3.jpg,maclura_pomifera,0
images/4.jpg,maclura_pomifera,0
...,...,...
images/18348.jpg,aesculus_glabra,107
images/18349.jpg,liquidambar_styraciflua,75
images/18350.jpg,cedrus_libani,133


## Cutomerize leaves Dataset

In [3]:
class Leaves_Dataset(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None):
        self.img_labels = pd.read_csv(annotations_file,index_col='image')
        self.img_dir = img_dir
        self.transform = transform
        
    def __len__(self):
        return len(self.img_labels)
    
    def __getitem__(self, index):
        img_path = self.img_dir + '/' + self.img_labels.index[index]
        image = Image.open(img_path)
        if self.transform:
            image = self.transform(image)
        label = self.img_labels.iloc[index, 1]
        return image, label

### Design transforms for augment

In [4]:
train_transform = transforms.Compose([
    # 随机裁剪图像，所得图像为原始面积的0.08到1之间，高宽比在3/4和4/3之间。
    # 然后，缩放图像以创建224 x 224的新图像
    transforms.RandomResizedCrop(224, scale=(0.5, 1.0), ratio=(3.0 / 4.0, 4.0 / 3.0)),
    transforms.RandomHorizontalFlip(),
    # 随机更改亮度，对比度和饱和度
    #transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4),
    # 添加随机噪声
    transforms.ToTensor(),
    # 标准化图像的每个通道
    transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])])

val_test_transform = transforms.Compose([
    transforms.Resize(256),
    # 从图像中心裁切224x224大小的图片
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])])

### Test Dataset

In [5]:
class Leaves_Predict_Dataset(Leaves_Dataset):
    def __getitem__(self, index):
        img_path = self.img_dir + '/' + self.img_labels.index[index]
        image = Image.open(img_path)
        if self.transform:
            image = self.transform(image)
        #label = self.img_labels.iloc[index, 1]
        return image

### Datasets and Loaders

In [6]:
train_dataset = Leaves_Dataset('../input/trainindexvs/trainindexs2.csv', root,transform=train_transform)
valid_dataset = Leaves_Dataset('../input/trainindexvs/trainindexs2.csv', root,transform=val_test_transform)
test_dataset = Leaves_Predict_Dataset('../input/d2lclassifyleaves/test.csv', root,transform=val_test_transform)

In [7]:
train_loader = DataLoader(dataset=train_dataset, batch_size=108, shuffle=True)
valid_loader = DataLoader(dataset=valid_dataset, batch_size=108, shuffle=True)
test_loader = DataLoader(dataset=test_dataset,shuffle=True)

# Inception_Resnet_blocks

In [8]:
class BasicConv2d(nn.Module):
    def __init__(self,in_channels,out_channels,kernel_size,x,stride=1,padding=0):
        super().__init__()
        self.conv2d = nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, stride=stride, padding=padding)
        self.bn = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU()
    def forward(self,x):
        return self.relu(self.bn(self.conv2d(x)))

In [9]:
class Inception_block_A(nn.Module):
    def __init__(self, scale=1.0):
        super().__init__()

        self.scale = scale
        
        self.preconv = BasicConv2d(256, 64, kernel_size=3,padding=1,x=56, stride=1)

        self.branch0 = BasicConv2d(64, 32, kernel_size=3,padding=1,x=56, stride=1)

        self.branch1 = nn.Sequential(
            BasicConv2d(64, 32, kernel_size=3,x=56, stride=1,padding=1),
            BasicConv2d(32, 32, kernel_size=3,x=56, stride=1, padding=1)
        )

        self.branch2 = nn.Sequential(
            BasicConv2d(64, 48, kernel_size=3,x=56, stride=1,padding=1),
            BasicConv2d(48, 48, kernel_size=3,x=56, stride=1,padding=1),
            BasicConv2d(48, 32, kernel_size=3,x=56, stride=1, padding=1),
        )

        self.conv2d = nn.Conv2d(96, 256, kernel_size=3,stride=1,padding=1)
        self.relu = nn.ReLU()

    def forward(self, x):
        px = self.preconv(x)
        x0 = self.branch0(px)
        x1 = self.branch1(px)
        x2 = self.branch2(px)
        out = torch.cat((x0, x1, x2), dim=1)
        out = self.conv2d(out)
        out = out * self.scale + x
        out = self.relu(out)
        return out

# Inception_Resnet_v2

In [10]:
class Inception_ResNet_v2(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        
        self.Inception_block1 = nn.Sequential(
            BasicConv2d(3, 32, kernel_size=3, padding=1,x=112,stride=2),
            BasicConv2d(32, 32, kernel_size=3, padding=1,x=112,stride=1),
            BasicConv2d(32, 64, kernel_size=3, padding=1,x=56,stride=2),
            BasicConv2d(64, 128, kernel_size=3, padding=1,x=56,stride=1),
            BasicConv2d(128, 256, kernel_size=3, padding=1, x=56,stride=1),
        )#[256, 56, 56]
        
        self.repeat_resnet_A = nn.Sequential(
            Inception_block_A(scale=0.3),
            Inception_block_A(scale=0.3),
            Inception_block_A(scale=0.3),
            Inception_block_A(scale=0.3),
            Inception_block_A(scale=0.3),
            Inception_block_A(scale=0.3),
            Inception_block_A(scale=0.3),
            Inception_block_A(scale=0.3),
        )
        
        self.pred_block = nn.Sequential(
            BasicConv2d(256, num_classes, kernel_size=1, x=56,stride=1),
            nn.AdaptiveAvgPool2d((1,1)),
            nn.Flatten()
        )#[batch,17]
        
        self.net = nn.Sequential(self.Inception_block1,
                                 self.repeat_resnet_A,
                                 self.pred_block
                                )
        
    def forward(self, x):
        return self.net(x)

In [11]:
inception_resnet_v2 = Inception_ResNet_v2(num_classes=176)

In [15]:
class Inception_ResNet_v3(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        
        self.Inception_block1 = nn.Sequential(
            BasicConv2d(3, 32, kernel_size=3, padding=1,x=112,stride=2),
            BasicConv2d(32, 32, kernel_size=3, padding=1,x=112,stride=1),
            BasicConv2d(32, 64, kernel_size=3, padding=1,x=56,stride=2),
            BasicConv2d(64, 128, kernel_size=3, padding=1,x=56,stride=1),
            BasicConv2d(128, 256, kernel_size=3, padding=1, x=56,stride=1),
        )#[256, 56, 56]
        
        self.repeat_resnet_A = nn.Sequential(
            Inception_block_A(scale=0.3),
            Inception_block_A(scale=0.3),
            Inception_block_A(scale=0.3),
            Inception_block_A(scale=0.3),
            Inception_block_A(scale=0.3),
            Inception_block_A(scale=0.3),
            Inception_block_A(scale=0.3),
            Inception_block_A(scale=0.3),
        )
        
        self.repeat_block_B = nn.Sequential(
            Inception_block_A(scale=0.3),
            Inception_block_A(scale=0.3),
            Inception_block_A(scale=0.3),
            Inception_block_A(scale=0.3),
            Inception_block_A(scale=0.3),
            Inception_block_A(scale=0.3),
            Inception_block_A(scale=0.3),
            Inception_block_A(scale=0.3),
            Inception_block_A(scale=0.3),
            Inception_block_A(scale=0.3),
            Inception_block_A(scale=0.3),
            Inception_block_A(scale=0.3),
        )
        
        self.pred_block = nn.Sequential(
            BasicConv2d(256, num_classes, kernel_size=1, x=56,stride=1),
            nn.AdaptiveAvgPool2d((1,1)),
            nn.Flatten()
        )#[batch,17]
        
        self.net = nn.Sequential(self.Inception_block1,
                                 self.repeat_resnet_A,
                                 self.repeat_block_B,
                                 self.pred_block
                                )
        
    def forward(self, x):
        return self.net(x)

In [17]:
inception_resnet_v3 = Inception_ResNet_v3(num_classes=176)

In [15]:
def layers_show(X, nn):
    '''To show the layers architecture of net'''
    for layer in nn.net:
        X = layer(X)
        print(layer.__class__.__name__,'Output:\t',X.shape)

In [16]:
'''X = torch.rand(size=(1,3,224,224))
layers_show(X, inception_resnet_v2)'''

'X = torch.rand(size=(1,3,224,224))\nlayers_show(X, inception_resnet_v2)'

# Train module

In [12]:
criterion = nn.CrossEntropyLoss()  # 损失函数

# 训练循环
def train(model, train_loader, criterion, device,lr, num_epochs=1):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    model.to(device)  # 将模型移动到指定的device上
    for epoch in range(num_epochs):
        model.train()  # 设置模型为训练模式
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)  # 将数据移动到指定的device上

            optimizer.zero_grad()  # 梯度清零
            outputs = model(images)  # 前向传播
            loss = criterion(outputs, labels)  # 计算损失
            loss.backward()  # 反向传播
            optimizer.step()  # 更新模型参数

            running_loss += loss.item()
            
            correct = 0
            total = 0
            with torch.no_grad():
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
                print(f'Epoch {epoch+1}, Batch Loss: {loss.item()}, Batch Accuracy: {100 * correct / total}%')
            
        print(f'Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}')
    torch.save(inception_resnet_v2.state_dict(),'/kaggle/working/Leaves_classifier7.pth')

# 评估循环
def evaluate(model, data_loader):
    model.eval()  # 设置模型为评估模式
    correct = 0
    total = 0
    with torch.no_grad():  # 不计算梯度
        for images, labels in data_loader:
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print(f'Accuracy: {100 * correct / total}%')

# 使用定义的函数进行训练和评估
# train(model, train_loader, criterion, optimizer)
# evaluate(model, test_loader)

In [23]:
inception_resnet_v2_state = torch.load('/kaggle/input/d2l_leavesclass/pytorch/class1/7/Leaves_classifier8.pth',map_location=torch.device('cpu')))
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

inception_resnet_v3_state = Inception_ResNet_v3.state_dict()
inception_resnet_v3_state.update({k: v for k, v in inception_resnet_v2_state.items() if k in inception_resnet_v3_state})
inception_resnet_v3.load_state_dict(inception_resnet_v3_state)
train(inception_resnet_v3, train_loader, criterion, num_epochs=3,lr=0.05, device=device)

SyntaxError: invalid syntax (3341449937.py, line 1)

In [30]:
train(inception_resnet_v2, train_loader, criterion, num_epochs=4,lr=0.05, device=device)

Epoch 1, Batch Loss: 3.679417371749878, Batch Accuracy: 11.11111111111111%
Epoch 1, Batch Loss: 3.868224620819092, Batch Accuracy: 13.88888888888889%
Epoch 1, Batch Loss: 3.846728801727295, Batch Accuracy: 10.185185185185185%
Epoch 1, Batch Loss: 3.824479818344116, Batch Accuracy: 10.185185185185185%
Epoch 1, Batch Loss: 3.7184391021728516, Batch Accuracy: 12.037037037037036%
Epoch 1, Batch Loss: 3.687220573425293, Batch Accuracy: 15.74074074074074%
Epoch 1, Batch Loss: 3.861595392227173, Batch Accuracy: 12.149532710280374%
Epoch 1, Loss: 3.783729382923671
Epoch 2, Batch Loss: 3.536346912384033, Batch Accuracy: 18.51851851851852%
Epoch 2, Batch Loss: 3.486783027648926, Batch Accuracy: 23.14814814814815%
Epoch 2, Batch Loss: 3.7278318405151367, Batch Accuracy: 13.88888888888889%
Epoch 2, Batch Loss: 3.8046019077301025, Batch Accuracy: 6.481481481481482%
Epoch 2, Batch Loss: 3.6764750480651855, Batch Accuracy: 17.59259259259259%
Epoch 2, Batch Loss: 3.8654043674468994, Batch Accuracy: 9.

In [ ]:
#torch.save(inception_resnet_v2.state_dict(),'/kaggle/working/Leaves_classifier3.pth')

In [25]:
import os
os.chdir('/kaggle/working')
print(os.getcwd())

print(os.listdir('/kaggle/working'))
from IPython.display import FileLink
FileLink('Leaves_classifier7.pth')

/kaggle/working
['Leaves_classifier7.pth', 'trainindex.csv']


/kaggle/working/Leaves_classifier7.pth